In [ ]:
# 2025-12-12 against production
from tqdm import tqdm
import pandas as pd
from django.core.exceptions import ObjectDoesNotExist
from apis_core.apis_metainfo.models import Collection, Uri
from normdata.utils import import_from_normdata
from AcdhArcheAssets.uri_norm_rules import get_normalized_uri 

In [ ]:
url = "https://raw.githubusercontent.com/arthur-schnitzler/burgtheater-event-crawler/refs/heads/main/data/persons_enriched.csv"
domain = "burgtheater"
base_url = "https://kulturerbe.burgtheater.at/person/"

In [ ]:
df = pd.read_csv(url)
df = df.dropna(subset=['gnd'])

In [ ]:
df.head()

In [ ]:
col, _ = Collection.objects.get_or_create(name="Burgtheater")

In [ ]:
data = []
for i, row in tqdm(df.iterrows(), total=len(df)):
    gnd_url = f"https://d-nb.info/gnd/{row["gnd"]}"
    gnd = get_normalized_uri(gnd_url)
    local_id = row["_id"]
    beacon_uri = f"{base_url}{row['_id']}"
    try:
        item = Uri.objects.get(uri=gnd)
        entity = item.entity
    except ObjectDoesNotExist:
        entity = import_from_normdata(gnd, "person")
    item = {
        "id": local_id,
        "pmb_id": entity.id
    }
    data.append(item)
    try:
        entity.collection.add(col)
    except AttributeError:
        print(gnd, beacon_uri)
        continue
    try:
        Uri.objects.get(uri=beacon_uri)
        continue
    except ObjectDoesNotExist:
        new_uri = Uri.objects.create(uri=beacon_uri, entity=entity)
        new_uri.domain = domain
        new_uri.save()